<a href="https://colab.research.google.com/github/JockWang/tfboy/blob/master/Tensorflow%E6%A8%A1%E5%9E%8B%E5%BB%BA%E7%AB%8B%E4%B8%8E%E8%AE%AD%E7%BB%83_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip install tensorflow-gpu==2.0

In [3]:
# 在Tensorflow中，推荐使用Keras（tf.keras）构建模型。Keras有两个重要的概念
# 模型（Model）和层（Layer）
# 层是对计算流程和变量进行封装，而模型是将各种层进行组织和连接，封装成一个整体
# Keras模型以类的形式呈现，通过继承tf.keras.Model
# 继承类中需要重写__init__()和call(input)两个方法
import tensorflow as tf
tf.__version__

'2.0.0'

In [6]:
# 一个简单的线性模型
X = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
y = tf.constant([[10.0], [20.0]])

class Linear(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.dense = tf.keras.layers.Dense(
        units=1,
        activation=None,
        kernel_initializer=tf.zeros_initializer(),
        bias_initializer=tf.zeros_initializer()
    )
  
  def call(self, input):
    output = self.dense(input)
    return output

model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
for i in range(100):
  with tf.GradientTape() as tape:
    y_pred = model(X)
    loss = tf.reduce_mean(tf.square(y_pred - y))
  grads = tape.gradient(loss, model.variables)
  optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
print(model.variables)

[<tf.Variable 'linear_1/dense_1/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[0.40784496],
       [1.191065  ],
       [1.9742855 ]], dtype=float32)>, <tf.Variable 'linear_1/dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.78322077], dtype=float32)>]


In [0]:
# 多层感知机MLP
import numpy as np

class MNISTLoader():
  def __init__(self):
    mnist = tf.keras.datasets.mnist
    (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
    self.train_data = np.expand_dims(self.train_data.astype(np.float32)/255.0, axis=-1)
    self.test_data = np.expand_dims(self.test_data.astype(np.float32)/255.0, axis=-1)
    self.train_label = self.train_label.astype(np.int32)
    self.test_label = self.test_label.astype(np.int32)
    self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
  def get_batch(self, batch_size):
    index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
    return self.train_data[index, :], self.train_label[index]

In [0]:
class MLP(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.flatten = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
    self.dense2 = tf.keras.layers.Dense(units=10)

  def call(self, inputs):
    x = self.flatten(inputs)
    x = self.dense1(x)
    x = self.dense2(x)
    output = tf.nn.softmax(x)
    return output

In [0]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001

In [0]:
model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [13]:
num_batches = int(data_loader.num_train_data//batch_size*num_epochs)
for batch_index in range(num_batches):
  X, y = data_loader.get_batch(batch_size)
  with tf.GradientTape() as tape:
    y_pred = model(X)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
    loss = tf.reduce_mean(loss)
    print('batch %d: loss %f'%(batch_index, loss.numpy()))
  grads = tape.gradient(loss, model.variables)
  optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 2.356072
batch 1: loss 2.363470
batch 2: loss 2.190720
batch 3: loss 2.176950
batch 4: loss 2.052395
batch 5: loss 2.064774
batch 6: loss 2.034676
batch 7: loss 1.768667
batch 8: loss 1.833820
batch 9: loss 1.802810
batch 10: loss 1.802572
batch 11: loss 1.701546
batch 12: loss 1.758663
batch 13: loss 1.713802
batch 14: loss 1.626154
batch 15: loss 1.482353
batch 16: loss 1.445636
batch 17: loss 1.509853
batch 18: loss 1.439431
batch 19: loss 1.212354
batch 20: loss 1.276268
batch 21: loss 1.320247
batch 22: loss 1.194940
batch 23: loss 1.272512
batch 24: loss 1.247144
batch 25: loss 1.000447
batch 26: loss 1.072712
batch 27: loss 1.151763
batch 28: loss 0.862842
batch 29: loss 0.963505
batch 30: loss 1.157526
batch 31: loss 1.031799
batch 32: loss 0.948137
batch 33: loss 1.101409
batch 34: loss 0.979469
batch 35: loss 0.854912
batch 36: loss 0.921572
batch 37: loss 0.941369
batch 38: loss 0.848238
batch 39: loss 0.907719
batch 40: loss 0.941799
batch 41: loss 0.780837
ba

In [14]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data//batch_size)
for batch_index in range(num_batches):
  start_index, end_index = batch_index * batch_size, (batch_index+1) * batch_size
  y_pred = model.predict(data_loader.test_data[start_index:end_index])
  sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index:end_index], y_pred=y_pred)
print('Test accuracy: %f'%sparse_categorical_accuracy.result())

Test accuracy: 0.974800
